In [22]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [23]:
import datetime

def fetch_latest_data():
    end_date = datetime.date.today().strftime('%Y-%m-%d')  # Gets the current date
    start_date = (datetime.date.today() - datetime.timedelta(days=730)).strftime('%Y-%m-%d')  # Approx. 2 years ago

    data = yf.download("ES=F", start=start_date, end=end_date, interval="1d")
    data = data.dropna()

    return data


In [24]:
# Data Preprocessing
def preprocess_data(data):
    # Exclude 'Close' column from normalization
    features = data.drop(columns=['Close'])

    # Normalize features
    scaler_features = MinMaxScaler()
    features_normalized = scaler_features.fit_transform(features)

    # Normalize 'Close' column separately
    scaler_close = MinMaxScaler()
    close_normalized = scaler_close.fit_transform(data[['Close']])

    # Combine normalized features and close column
    data_normalized = np.hstack([features_normalized, close_normalized])
    data = pd.DataFrame(data_normalized, columns=data.columns)

    # Outlier detection
    z_scores = np.abs(zscore(data))
    data = data[(z_scores < 3).all(axis=1)]

    return data, scaler_close


In [25]:
def feature_engineering(data):
    # Technical Indicators
    data['SMA_5'] = data['Close'].rolling(window=5).mean()
    data['SMA_10'] = data['Close'].rolling(window=10).mean()

    # Time Features
    day_of_week = []
    month = []
    for date_str in data.index:
        date_obj = pd.Timestamp(date_str)
        day_of_week.append(date_obj.weekday())
        month.append(date_obj.month)

    data['Day_of_Week'] = day_of_week
    data['Month'] = month

    # Rolling Statistics
    data['Rolling_Mean'] = data['Close'].rolling(window=5).mean()
    data['Rolling_Std'] = data['Close'].rolling(window=5).std()

    # Exponential Moving Average
    data['EMA_5'] = data['Close'].ewm(span=5, adjust=False).mean()
    data['EMA_10'] = data['Close'].ewm(span=10, adjust=False).mean()

    # Drop NaN values introduced by rolling computations
    data = data.dropna()

    return data


In [26]:
class Data:
    def __init__(self, features, prices):
        self.features = features
        self.prices = prices

class TreeNode:
    def __init__(self, feature_idx=None, threshold=None, left=None, right=None, gaussian_params=None):
        self.feature_idx = feature_idx
        self.threshold = threshold
        self.left = left
        self.right = right
        self.gaussian_params = gaussian_params

class DecisionNode(TreeNode):
    def __init__(self, feature_idx, threshold, data_size, left, right):
        super().__init__(feature_idx, threshold, left, right)
        self.data_size = data_size

class LeafNode(TreeNode):
    def __init__(self, gaussian_params):
        super().__init__(gaussian_params=gaussian_params)

In [27]:

def bayesian_update(prior_mean, prior_variance, likelihood_mean, likelihood_variance, data_size):
    epsilon = 1e-8  # small value to prevent division by zero

    # Prevent division by zero
    if likelihood_variance < epsilon:
        likelihood_variance = epsilon

    precision_prior = 1 / prior_variance
    precision_likelihood = data_size / likelihood_variance
    posterior_variance = 1 / (precision_prior + precision_likelihood)

    # Calculate the posterior mean
    posterior_mean = (prior_mean / prior_variance + likelihood_mean * data_size / likelihood_variance) * posterior_variance
    return posterior_mean, posterior_variance

def compute_likelihood(data, mean, variance):
    epsilon = 1e-8
    likelihoods = np.exp(-((data.prices - mean)**2) / (2 * (variance + epsilon))) / np.sqrt(2 * np.pi * (variance + epsilon))

    # Replace zeros with a small value
    likelihoods[likelihoods < epsilon] = epsilon
    return likelihoods


def hypothetical_split(data, feature_idx, threshold):
    left_mask = data.features[:, feature_idx] < threshold
    right_mask = ~left_mask
    left_data = Data(data.features[left_mask], data.prices[left_mask])
    right_data = Data(data.features[right_mask], data.prices[right_mask])
    return left_data, right_data

def bic_or_expected_utility(data, mean, variance):
    n = len(data.prices)
    log_likelihood = np.sum(np.log(compute_likelihood(data, mean, variance)))
    return log_likelihood - 1/2 * np.log(n)

def select_best_split(data, max_features=None):
    best_score = float('-inf')
    best_split = None
    n, m = data.features.shape
    if max_features is None:
        features_to_check = range(m)
    else:
        features_to_check = np.random.choice(m, max_features, replace=False)
    for feature_idx in features_to_check:
        thresholds = np.unique(data.features[:, feature_idx])
        for threshold in thresholds:
            left_data, right_data = hypothetical_split(data, feature_idx, threshold)
            if len(left_data.prices) == 0 or len(right_data.prices) == 0:
                continue
            prior_mean, prior_variance = 0, 1
            left_likelihood_mean, left_likelihood_variance = np.mean(left_data.prices), np.var(left_data.prices)
            right_likelihood_mean, right_likelihood_variance = np.mean(right_data.prices), np.var(right_data.prices)
            left_post_mean, left_post_var = bayesian_update(prior_mean, prior_variance, left_likelihood_mean, left_likelihood_variance, len(left_data.prices))
            right_post_mean, right_post_var = bayesian_update(prior_mean, prior_variance, right_likelihood_mean, right_likelihood_variance, len(right_data.prices))
            score = bic_or_expected_utility(left_data, left_post_mean, left_post_var) + bic_or_expected_utility(right_data, right_post_mean, right_post_var)
            if score > best_score:
                best_score = score
                best_split = (feature_idx, threshold)
    return best_split

def bayesian_decision_tree(data, max_depth, max_features=None):
    if max_depth == 0:
        likelihood_mean, likelihood_variance = np.mean(data.prices), np.var(data.prices)
        prior_mean, prior_variance = 0, 1
        post_mean, post_var = bayesian_update(prior_mean, prior_variance, likelihood_mean, likelihood_variance, len(data.prices))
        return LeafNode((post_mean, post_var))
    best_split = select_best_split(data, max_features)
    if best_split is None:
        likelihood_mean, likelihood_variance = np.mean(data.prices), np.var(data.prices)
        prior_mean, prior_variance = 0, 1
        post_mean, post_var = bayesian_update(prior_mean, prior_variance, likelihood_mean, likelihood_variance, len(data.prices))
        return LeafNode((post_mean, post_var))
    feature_idx, threshold = best_split
    left_data, right_data = hypothetical_split(data, feature_idx, threshold)
    left_child = bayesian_decision_tree(left_data, max_depth - 1, max_features)
    right_child = bayesian_decision_tree(right_data, max_depth - 1, max_features)
    return DecisionNode(feature_idx, threshold, len(data.prices), left_child, right_child)

def bootstrap(data, size=None):
    if size is None:
        size = len(data.prices)
    indices = np.random.choice(len(data.prices), size, replace=True)
    return Data(data.features[indices], data.prices[indices])

def random_bayesian_forest(data, n_trees, max_depth, max_features=None):
    trees = []
    for _ in range(n_trees):
        bootstrapped_data = bootstrap(data)
        tree = bayesian_decision_tree(bootstrapped_data, max_depth, max_features)
        trees.append(tree)
    return trees

def traverse_tree(node, sample):
    if node.gaussian_params:  # LeafNode
        mean, variance = node.gaussian_params
        return np.random.normal(mean, np.sqrt(variance))
    feature_idx, threshold = node.feature_idx, node.threshold
    if sample[feature_idx] < threshold:
        return traverse_tree(node.left, sample)
    else:
        return traverse_tree(node.right, sample)

def predict_forest(forest, sample):
    predictions = [traverse_tree(tree, sample) for tree in forest]
    return np.mean(predictions)

def evaluate_model(forest, test_data):
    predictions = [predict_forest(forest, sample) for sample in test_data.features]
    rmse = np.sqrt(mean_squared_error(test_data.prices, predictions))
    mae = mean_absolute_error(test_data.prices, predictions)
    return rmse, mae


In [28]:
def enhanced_feature_engineering(data):
    # Previous prices (lags)
    for i in range(1, 4):  # Add 3 lags
        data[f'lag_{i}'] = data['Close'].shift(i)

    # Rolling statistics
    data['rolling_mean_3'] = data['Close'].rolling(window=3).mean()
    data['rolling_std_3'] = data['Close'].rolling(window=3).std()

    data.dropna(inplace=True)  # Drop rows with NaN values due to lag and rolling features
    return data


In [29]:
def rolling_window_backtest(data, window_size, model_func):
    predictions = []
    actuals = []

    for end in range(window_size + 1, len(data) + 1):
        train_data = data.iloc[end - window_size - 1:end - 1]
        test_data = data.iloc[end - 1:end]

        X_train = train_data.drop(columns=['Close']).values
        y_train = train_data['Close'].values
        X_test = test_data.drop(columns=['Close']).values
        y_test = test_data['Close'].values

        forest = model_func(X_train, y_train)

        prediction = predict_forest(forest, X_test[0])

        predictions.append(prediction)
        actuals.append(y_test[0])

    return predictions, actuals



In [30]:
def train_rbf(X_train, y_train):

    train_data = Data(X_train, y_train)
    forest = random_bayesian_forest(train_data, n_trees=10, max_depth=3)
    return forest


In [31]:
def predict_price_for_today(data, window_size, scaler_close):
    """
    Predicts the price for today using the most recent data of size window_size.

    Parameters:
    - data: The processed data with features.
    - window_size: The size of the window (number of days) to use for training.
    - scaler_close: The MinMaxScaler object fitted to the 'Close' column.

    Returns:
    - Predicted price for today in original scale.
    """

    # Extract the most recent data
    latest_data = data.iloc[-window_size:]

    # Train the Random Bayesian Forest
    X_latest = latest_data.drop(columns=['Close']).values
    y_latest = latest_data['Close'].values
    forest = train_rbf(X_latest, y_latest)

    # Predict using today's features
    today_features = data.iloc[-1:].drop(columns=['Close']).values
    predicted_price_for_today = predict_forest(forest, today_features[0])

    # Convert to original scale
    predicted_price_for_today_original_scale = scaler_close.inverse_transform([[predicted_price_for_today]])[0][0]

    return predicted_price_for_today_original_scale


In [ ]:
# 1. Fetch the latest data
data = fetch_latest_data()

# 2. Preprocess and feature engineering
data, scaler_close = preprocess_data(data)
data = enhanced_feature_engineering(data)

# 3. Train-Test Split
#X = data.drop(columns=['Close']).values
#y = data['Close'].values
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
#train_data = Data(X_train, y_train)
#test_data = Data(X_test, y_test)

# 4. Model Training
#forest = random_bayesian_forest(train_data, n_trees=10, max_depth=3)

predictions, actuals = rolling_window_backtest(data, window_size=60, model_func=train_rbf)  # Assuming 60 days for training

rmse = np.sqrt(mean_squared_error(actuals, predictions))
mae = mean_absolute_error(actuals, predictions)
print(f"RMSE: {rmse}, MAE: {mae}")

predicted_today = predict_price_for_today(data, window_size=60, scaler_close=scaler_close)
print(f"Predicted price for today: {predicted_today}")

# 5. Model Evaluation
#rmse, mae = evaluate_model(forest, test_data)
#print(f"RMSE: {rmse}, MAE: {mae}")

# 6. Prediction for today
#X_latest = data.iloc[-1:].drop(columns=['Close']).values
#predicted_price = predict_forest(forest, X_latest[0])
#predicted_price_original_scale = scaler_close.inverse_transform([[predicted_price]])[0][0]
#print(f"Predicted price for today in original scale: {predicted_price_original_scale}")

#predicted_price_normalized = 0.7476125257069268  # This is just an example value
#predicted_price_original_scale = scaler_close.inverse_transform([[predicted_price_normalized]])[0][0]
#print(f"Predicted price for today in original scale: {predicted_price_original_scale}")



[*********************100%***********************]  1 of 1 completed


In [ ]:
!pip install graphviz


In [ ]:
import graphviz
from graphviz import Digraph

In [ ]:
def visualize_tree(node, feature_names=None, parent_name=None, graph=None, edge_label=None):
    if graph is None:
        graph = Digraph('BayesianTree', node_attr={'style': 'filled'})

    if isinstance(node, LeafNode):
        mean, variance = node.gaussian_params
        graph.node(name=str(id(node)),
                   label=f"Mean: {mean:.2f}\nVar: {variance:.2f}",
                   color='lightyellow')

    elif isinstance(node, DecisionNode):
        description = feature_names[node.feature_idx] if feature_names is not None else node.feature
        graph.node(name=str(id(node)),
                   label=f"{description} <= {node.threshold:.2f}\nSamples: {node.data_size}",
                   color='lightblue')

        # Left subtree (True branch)
        visualize_tree(node.left, feature_names, str(id(node)), graph, 'True')

        # Right subtree (False branch)
        visualize_tree(node.right, feature_names, str(id(node)), graph, 'False')

        if parent_name:
            graph.edge(parent_name, str(id(node)), label=edge_label)
    else:
        print(f"Unexpected node type: {type(node)}")
        print(node.__dict__)  # print attributes of the node

    return graph


In [ ]:
# Example usage
#tree_graph = visualize_tree(forest[0], feature_names=data.columns[:-1])
#tree_graph.view("Bayesian_Tree")

tree_graph = visualize_tree(forest[0], feature_names=data.columns[:-1])
tree_graph